In [1]:
!pip install tweepy
!pip install seaborn
!pip install plotly
!pip install jsonpickle
!pip install textblob
!pip install gensim
!pip install bokeh
!pip install configparser

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import tweepy
import configparser
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import jsonpickle
from textblob import TextBlob
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import os
import re
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk import ngrams, FreqDist
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import string
import json

nltk.download("vader_lexicon")

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...


### Read twitter credentials

In [3]:
config = configparser.RawConfigParser()
config.read("config/config.ini")
config.sections()

['twitter']

In [4]:
auth = tweepy.OAuthHandler(config["twitter"]["API_KEY"], config["twitter"]["API_SECRET_KEY"])
auth.set_access_token(config["twitter"]["ACCESS_TOKEN"], config["twitter"]["ACCESS_TOKEN_SECRET"])
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Helper Functions

In [5]:
class Tweet():
    def __init__(self, id, df):
        self.id = id
        self.url = self.get_tweet_url(id)
        self.full_text = df[df["id"] == id]["full_text"].values[0]
        self.created_at = df[df["id"] == id]["created_at"].values[0]

    def get_tweet_url(self, id):
        return "https://twitter.com/ex/status/" + str(id)

    def print(self):
        print("{} | Created at: {}\n{}\n\n{}".format(self.id, self.created_at, self.full_text, self.url))

# t = Tweet(1252383850086318084, tweets_clean)
# t.url


# example = Tweet(1252383850086318084, tweets_clean).print()

In [6]:
def clean_tweet(text):
    if text:
        """Remove emojis, retweet/mention tags, URLs and escape sequences."""
        text = text.encode('ascii', 'ignore').decode('ascii')
        text = re.sub("RT @[\w]*:","",text)
        text = re.sub("@[\w]*","",text)
        text = re.sub("https?://[A-Za-z0-9./]*","", text)
        text = re.sub("\n","",text)
    else:
        text = ""
    return text

### Set Up Search Query

* Tweet mode `extended` enables us to access tweets that contain more than 140 characters

In [7]:
TRACK_WORD = "Bermuda"

SEARCH_QUERY      = TRACK_WORD + " -filter:retweets"
SEARCH_LANG       = "en"
SEARCH_TWEET_MODE = "extended"

BATCH_SIZE        = 100 # 100 per request
MAX_TWEETS        = 100000

FILE_NAME         = "data/tweets_apr26_may04.txt"

FONT_STYLES       = dict(family='Oswald', color='#eeeeee')

COLORS            = ['mediumturquoise'] # https://developer.mozilla.org/en-US/docs/Web/CSS/color_value

In [8]:
def load_tweets(new_api_request=False):
    """
    Load tweets either via a new API request or from file.
    
    :param new_api_request: bool
    
    :returns: Dataframe
    """

    min_id = None
    max_id = -1

    if new_api_request:
        print("Making an API request...")
        num_tweets_fetched = 0
        with open(FILE_NAME, 'w') as f:
            while num_tweets_fetched < MAX_TWEETS:
                try:
                    if (max_id <= 0):
                        if (not min_id):
                            new_tweets = api.search(
                                q = SEARCH_QUERY,
                                count = BATCH_SIZE,
                                lang = SEARCH_LANG,
                                tweet_mode = SEARCH_TWEET_MODE)
                        else:
                            new_tweets = api.search(
                                q = SEARCH_QUERY,
                                count = BATCH_SIZE,
                                lang = SEARCH_LANG,
                                tweet_mode = SEARCH_TWEET_MODE,
                                since_id = min_id)
                    else:
                        if (not min_id):
                            new_tweets = api.search(
                                q = SEARCH_QUERY,
                                count = BATCH_SIZE,
                                lang = SEARCH_LANG,
                                tweet_mode = SEARCH_TWEET_MODE,
                                max_id = str(max_id - 1))
                        else:
                            new_tweets = api.search(
                                q = SEARCH_QUERY,
                                count = BATCH_SIZE,
                                lang = SEARCH_LANG,
                                tweet_mode = SEARCH_TWEET_MODE,
                                max_id = str(max_id - 1),
                                since_id = min_id)

                    if not new_tweets:
                        print("No tweets found.")
                        break
                    else:
                        for tweet in new_tweets:
                            f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
                        num_tweets_fetched += len(new_tweets)
                        max_id = new_tweets[-1].id
                        if num_tweets_fetched % 1000 == 0:
                            print("Downloaded {} tweets.".format(num_tweets_fetched))

                except tweepy.TweepError as e:
                    print("Tweepy error: " + str(e))
                    break

    if not os.path.exists(FILE_NAME):
        return pd.DataFrame()
    else:
        print("Loading from file...")
        return pd.read_json(path_or_buf=FILE_NAME, lines=True)

In [9]:
tweets_ = load_tweets(False)
tweets_.head()

Loading from file...
<string>:6: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compatibility.



,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities
0,2020-05-04 21:55:11+00:00,1257428586081943555,1257428586081943552,My answer to What would a biblically accurate ...,False,"[0, 145]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.quora.com/"" rel=""nofollow""...",NaN,NaN,NaN,NaN,None,"{'id': 118116355, 'id_str': '118116355', 'name...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
1,2020-05-04 21:53:11+00:00,1257428081251356672,1257428081251356672,•Bermuda•in Baby Blue 🏝 as we design for our s...,False,"[0, 241]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN,NaN,NaN,None,"{'id': 1723795837, 'id_str': '1723795837', 'na...","{'type': 'Point', 'coordinates': [26.1358, -80...","{'type': 'Point', 'coordinates': [-80.1418, 26...","{'id': '6c686af766d8429c', 'url': 'https://api...",NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
2,2020-05-04 21:50:21+00:00,1257427368311951363,1257427368311951360,I was hella focused on his hair and jawline th...,False,"[0, 122]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 1181247029395296256, 'id_str': '1181247...",None,None,None,NaN,True,0,0,False,False,0.0,en,1.257422e+18,1.257422e+18,{'created_at': 'Mon May 04 21:30:53 +0000 2020...,NaN
3,2020-05-04 21:47:21+00:00,1257426613874110470,1257426613874110464,Check out what I just added to my closet on Po...,False,"[0, 158]","{'hashtags': [{'text': 'shopmycloset', 'indice...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://poshmark.com/"" rel=""nofollow"">...",NaN,NaN,NaN,NaN,None,"{'id': 3320811722, 'id_str': '3320811722', 'na...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
4,2020-05-04 21:46:57+00:00,1257426514418831360,1257426514418831360,My mom and I in Bermuda 2000 celebrating my 3r...,False,"[0, 70]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 1250109452964986884, 'id_str': '1250109...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,"{'media': [{'id': 1257426512321617920, 'id_str..."


In [10]:
len(tweets_)

5681

In [11]:
import json

def get_most_recent_tweet_id_fetched(filename):
    with open(filename) as f:
        first_line = json.loads(f.readline())
        print(first_line["id"])

get_most_recent_tweet_id_fetched(FILE_NAME)


1257428586081943555


### Data Dictionary

* `created_at`: datetime of tweet creation
* `id` and `id_str`: unique identifiers that can be used to generate a URL pointing to the tweet
* `full_text`: content of text (retrieved via `extended` tweet mode)
* `truncated`: not relevant for extended tweet payload as no tweets will have been truncated
* `display_text_range`: delineates sections of tweet text (ex: where does a mention end and the body begin)
* `entities`: a group of fields containing all sub-elements of a tweet and their positions (ex: hashtags used and their start/end indices)
* `source`: origin of tweet in href format (ex: iphone, mobile web, web)
* `in_reply_to`: group of fields relating to the tweet/user being replied to
* `user.x`: group of fields relating to the user who created the tweet

### Cleaning Tweets

* Remove emojis, reweets/mentions, URLs and escape characters from text
* Convert `created_at` to datetime

In [13]:
tweets_clean = tweets_.copy()
tweets_clean["full_text"] = tweets_clean["full_text"].apply(clean_tweet)
tweets_clean["created_at"] = tweets_clean["created_at"].apply(pd.to_datetime)
tweets_clean.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,is_quote_status,retweet_count,favorite_count,favorited,retweeted,possibly_sensitive,lang,quoted_status_id,quoted_status_id_str,quoted_status,extended_entities
0,2020-05-04 21:55:11+00:00,1257428586081943555,1257428586081943552,My answer to What would a biblically accurate ...,False,"[0, 145]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.quora.com/"" rel=""nofollow""...",NaN,NaN,NaN,NaN,None,"{'id': 118116355, 'id_str': '118116355', 'name...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
1,2020-05-04 21:53:11+00:00,1257428081251356672,1257428081251356672,Bermudain Baby Blue as we design for our summ...,False,"[0, 241]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN,NaN,NaN,None,"{'id': 1723795837, 'id_str': '1723795837', 'na...","{'type': 'Point', 'coordinates': [26.1358, -80...","{'type': 'Point', 'coordinates': [-80.1418, 26...","{'id': '6c686af766d8429c', 'url': 'https://api...",NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
2,2020-05-04 21:50:21+00:00,1257427368311951363,1257427368311951360,I was hella focused on his hair and jawline th...,False,"[0, 122]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,NaN,None,"{'id': 1181247029395296256, 'id_str': '1181247...",None,None,None,NaN,True,0,0,False,False,0.0,en,1.257422e+18,1.257422e+18,{'created_at': 'Mon May 04 21:30:53 +0000 2020...,NaN
3,2020-05-04 21:47:21+00:00,1257426613874110470,1257426613874110464,Check out what I just added to my closet on Po...,False,"[0, 158]","{'hashtags': [{'text': 'shopmycloset', 'indice...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://poshmark.com/"" rel=""nofollow"">...",NaN,NaN,NaN,NaN,None,"{'id': 3320811722, 'id_str': '3320811722', 'na...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,NaN
4,2020-05-04 21:46:57+00:00,1257426514418831360,1257426514418831360,My mom and I in Bermuda 2000 celebrating my 3r...,False,"[0, 70]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,NaN,None,"{'id': 1250109452964986884, 'id_str': '1250109...",None,None,None,NaN,False,0,0,False,False,0.0,en,NaN,NaN,NaN,"{'media': [{'id': 1257426512321617920, 'id_str..."


In [14]:
users = tweets_clean["user"]
locations = [user["location"] for user in users]

### Apply TextBlob & NLTK for basic sentiment analysis

In [22]:
# TextBlob
tweets_clean["textblob_polarity"] = tweets_clean["full_text"].apply(lambda text: TextBlob(text).sentiment[0])
tweets_clean["textblob_subjectivity"] = tweets_clean["full_text"].apply(lambda text: TextBlob(text).sentiment[1])
tweets_clean["textblob_polarity_flag"] = ((tweets_clean["textblob_polarity"] > 0 ) * 1 + (tweets_clean["textblob_polarity"] < 0 ) * -1)

# NLTK
tweets_clean["nltk_polarity_scores"] = tweets_clean["full_text"].apply(lambda text: SentimentIntensityAnalyzer().polarity_scores(text))
tweets_clean["nltk_polarity"] = tweets_clean["full_text"].apply(lambda text: SentimentIntensityAnalyzer().polarity_scores(text)["compound"])
tweets_clean["nltk_polarity_flag"] = ((tweets_clean["nltk_polarity"] > 0 ) * 1 + (tweets_clean["nltk_polarity"] < 0 ) * -1)


### Save clean tweets

In [46]:
# tweets_clean.to_csv("tweets_clean.csv", index=False)

### Time series visualization

In [23]:
print("Earliest tweet: {} \nMost recent tweet: {}\nTime Elapsed: {}".format(tweets_clean["created_at"].min(), tweets_clean["created_at"].max(), tweets_clean["created_at"].max() - tweets_clean["created_at"].min()))

Earliest tweet: 2020-04-26 08:38:47+00:00 
Most recent tweet: 2020-05-04 21:55:11+00:00
Time Elapsed: 8 days 13:16:24


In [40]:
TIME_WINDOW = "6H"

time_series = tweets_clean[["id", "nltk_polarity_flag", "created_at"]].groupby([pd.Grouper(key="created_at", freq=TIME_WINDOW), "nltk_polarity_flag"]).count()
time_series = time_series.reset_index().rename(columns={"id":"count"})

fig = px.line(time_series, x="created_at", y="count", color="nltk_polarity_flag", color_discrete_sequence=['deeppink','white','greenyellow'])
fig.update_layout(
    title="Tweet Frequency & Polarity",
    xaxis_title="Time",
    yaxis_title="Count",
    template="plotly_dark",
    font=FONT_STYLES,
    legend_title="Polarity Flag"
)

fig.show()

### Sentiment analysis visualizations

In [41]:
fig = make_subplots(
        rows=1, cols=2,
        column_widths=[1, 1],
        row_heights=[2],
        y_title= "Count"
        )

fig.add_trace(go.Histogram(x=tweets_clean["textblob_polarity"],
    name="Polarity", marker=dict(color = 'aquamarine'), xbins=dict(start=np.min(tweets_clean["textblob_polarity"]), 
    size=0.2, end=np.max(tweets_clean["textblob_polarity"]))
    ), row=1, col=1)

fig.add_trace(go.Histogram(x=tweets_clean["textblob_subjectivity"],
    name="Subjectivity", marker=dict(color = 'lightsalmon'), xbins=dict(start=np.min(tweets_clean["textblob_polarity"]), 
    size=0.1, end=np.max(tweets_clean["textblob_polarity"]))
    ), row=1, col=2)

fig.update_xaxes(title_text="Polarity", row=1, col=1)
fig.update_xaxes(title_text="Subjectivity", row=1, col=2)

fig.update_layout(
    title_text = "Tweet Polarity & Subjectivity Using TextBlob Library",
    template="plotly_dark",
    font=FONT_STYLES
)
fig.show()

In [42]:
fig = px.scatter(tweets_clean, x="textblob_subjectivity", y="textblob_polarity", color_discrete_sequence=['greenyellow'])
fig.update_layout(
    title="Tweet Polarity vs. Subjectivity",
    xaxis_title="Subjectivity",
    yaxis_title="Polarity",
    template="plotly_dark",
    font=FONT_STYLES,
    yaxis = dict(
        tickmode = 'linear',
        tick0 = 0,
        dtick = 1
    )
)

fig.show()

In [44]:
fig = go.Figure()

x = np.arange(-1,1,0.1)
fig.add_trace(go.Scatter(x=x, y=x, line=dict(color="white"), name="1:1 Diagonal"))
fig.add_trace(go.Scatter(x=tweets_clean["nltk_polarity"], y=tweets_clean["textblob_polarity"], name="Tweet Sentiment Data", mode="markers", marker_color="fuchsia"))

fig.update_traces(marker_line_width=2, marker_size=10)
fig.update_xaxes(title_text="NLTK Polarity")
fig.update_yaxes(title_text="TextBlob Polarity")

fig.update_layout(
    title_text = "Comparison of Tweet Polarity Using TextBlob vs. NLTK",
    template="plotly_dark",
    font=FONT_STYLES
)

fig.show()
